# GraphRAG with Neo4j and LangChain and Gemini on VertexAI Reasoning Engine

This is a demonstration of a GeNAI API with advanced RAG patterns combining vector and graph search.

It is deployed on Vertex AI Reasoning Engine (Preview) as scalable infrastructure and can then be integrated with GenAI applications on Cloud Run via REST APIs.

## Dataset

The dataset is a graph about companies, associated industries, and people and articles that report on those companies.

![Graph Model](https://i.imgur.com/lWJZSEe.png)

The articles are chunked and the chunks also stored in the graph.

Embeddings are computed for each of the text chunks with `textembedding-gecko` (786 dim) and stored on each chunk node.
A Neo4j vector index `news_google` and a fulltext index `news_fulltext` (for hybrid search) were created.

The database is publicly available with a readonly user:

https://demo.neo4jlabs.com:7473/browser/

* URI: neo4j+s://demo.neo4jlabs.com
* User: companies
* Password: companies
* Companies: companies

We utilize the Neo4jVector LangChain integration, which allows for advanced RAG patterns.
We will utilize both hybrid search as well as parent-child retrievers and GraphRAG (extract relevant context).

In our configuration we provide both the vector and fulltext index as well as a retrieval query that fetches the following additional information for each chunk

* Parent `Article` of the `Chunk` (aggregate all chunks for a single article)
* `Organization`(s) mentioned
* `IndustryCategory`(ies) for the organization
* `Person`(s) connected to the organization and their roles (e.g. investor, chairman, ceo)

We will retrieve the top-k = 5 results from the vector index.

As LLM we will utilize Vertex AI *Gemini Pro 1.0*

We use a temperature of 0.1, top-k=40, top-p=0.8

Our `LangchainCode` class contains the methods for initialization which can only hold serializable information (strings and numbers).

In `set_up()` Gemini as LLM, VertexAI Embeddings and the `Neo4jVector` retriever are combined into a LangChain chain.

Which is then used in `query`  with `chain.invoke()`.

The class is deployed as ReasoningEngine with the Google Vertex AI Python SDK.
For the deployment you provide the instance of the class which captures relevant environment variables and configuration and the dependencies, in our case `google-cloud-vertexai, langchain, langchain_google_vertexai, neo4j`.

And after successful deploymnet we can use the resulting object via the `query` method, passing in our user question.

In [37]:
PROJECT_ID = "iamtests-315719"
REGION = "us-central1"
STAGING_BUCKET = "gs://neo4j-vertex-ai-extension2"


In [38]:
# from google.colab import auth
# auth.authenticate_user(project_id=PROJECT_ID)

!gcloud config set project $PROJECT_ID

Updated property [core/project].


In [ ]:
!pip install --quiet neo4j==5.19.0
!pip install --quiet langchain_google_vertexai==1.0.4
!pip install --quiet --force-reinstall langchain==0.2.0 langchain_community==0.2.0


In [5]:
!pip install --quiet google-cloud-aiplatform==1.51.0
!pip install --quiet  google-cloud-resource-manager==1.12.3

In [39]:
import vertexai
from vertexai.preview import reasoning_engines

vertexai.init(
    project=PROJECT_ID,
    location=REGION,
    staging_bucket=STAGING_BUCKET,
)

In [40]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
from langchain_google_vertexai import ChatVertexAI, VertexAIEmbeddings
from langchain_community.vectorstores import Neo4jVector
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough


If you installed any packages above, you can restart the runtime to pick them up:

Click on the "Runtime" button at the top of Colab.
Select "Restart session".
You would not need to re-run the cell above this one (to reinstall the packages).

In [67]:
import os
from typing import Callable, Sequence

URI = os.getenv('NEO4J_URI', 'neo4j+s://demo.neo4jlabs.com')
USER = os.getenv('NEO4J_USERNAME','companies')
PASSWORD = os.getenv('NEO4J_PASSWORD','companies')
DATABASE = os.getenv('NEO4J_DATABASE','companies')

class LangchainCode:
    def __init__(
            self,
            model: str,
            tools: Sequence[Callable],
        ):
        self.model_name = model #"gemini-pro"
        self.max_output_tokens = 1024
        self.temperature = 0.1
        self.top_p = 0.8
        self.top_k = 40
        self.project_id = PROJECT_ID
        self.location = REGION
        self.tools = tools
        self.uri = URI
        self.username = USER
        self.password = PASSWORD
        self.database = DATABASE
        self.prompt_input_variables = ["query"]
        self.prompt_template="""
            You are a venture capital assistant that provides useful answers about companies, their boards, financing etc.
            only using the information from a company database already provided in the context.
            Prefer higher rated information in your context and add source links in your answers.
            Context: {context}"""

    def set_up(self):
        from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
        from langchain_google_vertexai import VertexAIEmbeddings, ChatVertexAI
        from langchain_community.vectorstores import Neo4jVector
        from langchain_core.output_parsers import StrOutputParser
        from langchain_core.runnables import RunnableParallel, RunnablePassthrough

        # agents module
        from langchain_google_vertexai import ChatVertexAI
        from langchain.agents import AgentExecutor
        from langchain.agents.format_scratchpad.tools import format_to_tool_messages
        from langchain.agents.output_parsers.tools import ToolsAgentOutputParser
        from langchain.tools.base import StructuredTool
        from langchain_core import prompts

        llm = ChatVertexAI(
            model_name=self.model_name,
            max_output_tokens=self.max_output_tokens,
            max_input_tokens=32000,
            temperature=self.temperature,
            top_p=self.top_p,
            top_k=self.top_k,
            project = self.project_id,
            location = self.location,
            # convert_system_message_to_human=True,
            response_validation=False,
            verbose=True
        )
        if self.tools:
            llm = llm.bind_tools(tools=self.tools)

        embeddings = VertexAIEmbeddings("textembedding-gecko@001")
        
        # agent block
        prompt = {
            "input": lambda x: x["input"],
            "agent_scratchpad": (
                lambda x: format_to_tool_messages(x["intermediate_steps"])
            ),
        } | prompts.ChatPromptTemplate.from_messages([
            ("user", "{input}"),
            prompts.MessagesPlaceholder(variable_name="agent_scratchpad"),
        ])
        self.agent_executor = AgentExecutor(
            agent=prompt | llm | ToolsAgentOutputParser(),
            tools=[StructuredTool.from_function(tool) for tool in self.tools],
        )

        # self.qa_chain = self.configure_qa_rag_chain(llm, embeddings)

    def query(self, input: str):
        '''Query the application.

        Args:
            input: The user prompt.

        Returns:
            The output of querying the application with the given input.
        '''
        # return self.qa_chain.invoke(query)
        return self.agent_executor.invoke(input={"input": input})

In [71]:
# create this function as a tool
def configure_qa_rag_chain(self, llm, embeddings):
    '''
    docstring
    '''
    qa_prompt = ChatPromptTemplate.from_messages([
        SystemMessagePromptTemplate.from_template(self.prompt_template),
        HumanMessagePromptTemplate.from_template("Question: {question}"
                                                    "\nWhat else can you tell me about it?"),
    ])

    # Vector + Knowledge Graph response
    kg = Neo4jVector.from_existing_index(
        embedding=embeddings,
        url=self.uri, username=self.username, password=self.password,database=self.database,
        search_type="hybrid",
        keyword_index_name="news_fulltext",
        index_name="news_google",
        retrieval_query="""
            WITH node as c,score
            MATCH (c)<-[:HAS_CHUNK]-(article:Article)

            WITH article, collect(distinct c.text) as texts, avg(score) as score
            RETURN article {.title, .sentiment, .siteName, .summary,
                organizations: [ (article)-[:MENTIONS]->(org:Organization) |
                        org { .name, .revenue, .nbrEmployees, .isPublic, .motto, .summary,
                        orgCategories: [ (org)-[:HAS_CATEGORY]->(i) | i.name],
                        people: [ (org)-[rel]->(p:Person) | p { .name, .summary, role: replace(type(rel),"HAS_","") }]}],
                texts: texts} as text,
            score, {source: article.siteName} as metadata
        """,
    )
    retriever = kg.as_retriever(search_kwargs={"k": 5})

    def format_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)

    chain = (
        {"context": retriever | format_docs , "question": RunnablePassthrough()}
        | qa_prompt
        | llm
        | StrOutputParser()
    )
    return chain

In [72]:
from langchain.globals import set_debug
set_debug(False)


In [73]:
# testing locally
lc = LangchainCode(
    model="gemini-1.5-pro-preview-0409",
    tools=[configure_qa_rag_chain]
)
lc.set_up()

response = lc.query('What are the news about IBM and its acquisitions and who are the people involved?')
print(response)

Retrying langchain_google_vertexai.chat_models._completion_with_retry.<locals>._completion_with_retry_inner in 4.0 seconds as it raised InvalidArgument: 400 Unable to submit request because one or more function parameters didn't specify the schema type field. Learn more: https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/function-calling.
Retrying langchain_google_vertexai.chat_models._completion_with_retry.<locals>._completion_with_retry_inner in 4.0 seconds as it raised InvalidArgument: 400 Unable to submit request because one or more function parameters didn't specify the schema type field. Learn more: https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/function-calling.
Retrying langchain_google_vertexai.chat_models._completion_with_retry.<locals>._completion_with_retry_inner in 4.0 seconds as it raised InvalidArgument: 400 Unable to submit request because one or more function parameters didn't specify the schema type field. Learn more: https://cloud.goog

InvalidArgument: 400 Unable to submit request because one or more function parameters didn't specify the schema type field. Learn more: https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/function-calling

In [16]:
remote_app = reasoning_engines.ReasoningEngine.create(
    LangchainCode(),
    requirements=[
        "google-cloud-aiplatform==1.51.0",
        "langchain_google_vertexai==1.0.4",
        "langchain==0.2.0",
        "langchain_community==0.2.0",
        "neo4j==5.19.0"
    ],
    display_name="Neo4j Vertex AI RE Companies",
    description="Neo4j Vertex AI RE Companies",
    sys_version="3.10",
    extra_packages=[]
)

INFO:vertexai.reasoning_engines._reasoning_engines:Using bucket neo4j-vertex-ai-extension
INFO:vertexai.reasoning_engines._reasoning_engines:Writing to gs://neo4j-vertex-ai-extension/reasoning_engine/reasoning_engine.pkl
INFO:vertexai.reasoning_engines._reasoning_engines:Writing to gs://neo4j-vertex-ai-extension/reasoning_engine/requirements.txt
INFO:vertexai.reasoning_engines._reasoning_engines:Creating in-memory tarfile of extra_packages
INFO:vertexai.reasoning_engines._reasoning_engines:Writing to gs://neo4j-vertex-ai-extension/reasoning_engine/dependencies.tar.gz
INFO:vertexai.reasoning_engines._reasoning_engines:Creating ReasoningEngine
INFO:vertexai.reasoning_engines._reasoning_engines:Create ReasoningEngine backing LRO: projects/990868019953/locations/us-central1/reasoningEngines/7289005628154970112/operations/3531746006863446016
INFO:vertexai.reasoning_engines._reasoning_engines:ReasoningEngine created. Resource name: projects/990868019953/locations/us-central1/reasoningEngines

In [17]:
response = remote_app.query(query="Who is on the board of Siemens?")
print(response)

The board of Siemens is composed of:

* Jim Hagemann Snabe (Manager and chairman) [source](https://automation.com/story/184274/siemens-brings-real-time-supply-chain-intelligence-to-siemens-xcelerator-and-the-digital-twin)
* Dominika Bettman (CEO at Siemens) [source](https://automation.com/story/184274/siemens-brings-real-time-supply-chain-intelligence-to-siemens-xcelerator-and-the-digital-twin)
* Alejandro Preinfalk (CEO at Siemens) [source](https://automation.com/story/184274/siemens-brings-real-time-supply-chain-intelligence-to-siemens-xcelerator-and-the-digital-twin)
* Miguel Angel Lopez Borrego (CEO at Siemens) [source](https://automation.com/story/184274/siemens-brings-real-time-supply-chain-intelligence-to-siemens-xcelerator-and-the-digital-twin)
* Hanna Hennig (CIO at Siemens) [source](https://automation.com/story/184274/siemens-brings-real-time-supply-chain-intelligence-to-siemens-xcelerator-and-the-digital-twin)
* Barbara Humpton (CEO at Siemens Bank) [source](https://automati